In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torchinfo import summary


In [2]:
class Classifier(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.Tanh(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.seq(x)


In [3]:
RANDOM_SEED = 42
NUM_FEATURES = 20
NUM_SAMPLES = 1000
BATCH_SIZE = 32
EPOCHS = 15

X, y = make_classification(
    n_samples=NUM_SAMPLES,
    n_features=NUM_FEATURES,
    n_informative=15,
    n_redundant=2,
    random_state=RANDOM_SEED
)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED
)



In [5]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Classifier(NUM_FEATURES).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)


In [7]:
summary(model, input_size=(BATCH_SIZE, NUM_FEATURES))


Layer (type:depth-idx)                   Output Shape              Param #
Classifier                               [32, 1]                   --
├─Sequential: 1-1                        [32, 1]                   --
│    └─Linear: 2-1                       [32, 64]                  1,344
│    └─ReLU: 2-2                         [32, 64]                  --
│    └─Dropout: 2-3                      [32, 64]                  --
│    └─Linear: 2-4                       [32, 32]                  2,080
│    └─ReLU: 2-5                         [32, 32]                  --
│    └─Linear: 2-6                       [32, 16]                  528
│    └─Tanh: 2-7                         [32, 16]                  --
│    └─Linear: 2-8                       [32, 1]                   17
│    └─Sigmoid: 2-9                      [32, 1]                   --
Total params: 3,969
Trainable params: 3,969
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.13
Input size (MB): 0.00
Forward/backward p

In [9]:
print(f"Training on {device}")
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
            
            predicted = (outputs > 0.5).float()
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
    
    # Update scheduler
    scheduler.step(val_loss)
    
    # Metrics
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(test_loader.dataset)
    accuracy = 100 * correct / total
    
    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Accuracy: {accuracy:.2f}%")



Training on cuda
Epoch 1/15 | Train Loss: 0.0840 | Val Loss: 0.1359 | Accuracy: 94.50%
Epoch 2/15 | Train Loss: 0.0989 | Val Loss: 0.1328 | Accuracy: 94.50%
Epoch 3/15 | Train Loss: 0.0901 | Val Loss: 0.1224 | Accuracy: 96.00%
Epoch 4/15 | Train Loss: 0.0903 | Val Loss: 0.1322 | Accuracy: 95.00%
Epoch 5/15 | Train Loss: 0.0734 | Val Loss: 0.1276 | Accuracy: 95.00%
Epoch 6/15 | Train Loss: 0.0798 | Val Loss: 0.1148 | Accuracy: 95.00%
Epoch 7/15 | Train Loss: 0.0645 | Val Loss: 0.1167 | Accuracy: 95.00%
Epoch 8/15 | Train Loss: 0.0856 | Val Loss: 0.1210 | Accuracy: 95.50%
Epoch 9/15 | Train Loss: 0.0578 | Val Loss: 0.1171 | Accuracy: 95.50%
Epoch 10/15 | Train Loss: 0.0690 | Val Loss: 0.1141 | Accuracy: 95.50%
Epoch 11/15 | Train Loss: 0.0748 | Val Loss: 0.1104 | Accuracy: 95.50%
Epoch 12/15 | Train Loss: 0.0542 | Val Loss: 0.1109 | Accuracy: 95.50%
Epoch 13/15 | Train Loss: 0.0539 | Val Loss: 0.1111 | Accuracy: 95.50%
Epoch 14/15 | Train Loss: 0.0655 | Val Loss: 0.1112 | Accuracy: 95.50